In [7]:
import pandas as pd


company_df = pd.read_csv('raw_data/company_info.csv')
people_df = pd.read_csv('raw_data/people_info.csv')
event_df = pd.read_csv('raw_data/event_info.csv')


In [8]:
from datetime import datetime
import re

def revenue_cleaning(value):
    if pd.isna(value):
        return 0
    value = str(value)

    value = value.replace('$', '').replace(' million', '000000').replace(' billion', '000000000')

    value =  float(value)

    return value

# company revenue
company_df['company_revenue'] = company_df['company_revenue'].apply(revenue_cleaning)


# Function to extract min and max employee counts
def extract_employee_counts(value):
    if pd.isna(value):
        return None, None
    value = str(value)
    
    # Remove 'employees' text
    value = value.replace(' employees', '').replace('Employees', '').strip()
    # Handle ranges (e.g., '51-200')
    if '-' in value:
        parts = value.split('-')
        min_count = int(parts[0].replace(',', '').strip())
        max_count = int(parts[1].replace(',', '').strip())
    else:
        value = re.sub(r'\..*', '', value)
        min_count = max_count = int(value.replace(',', '').replace('+', '').strip())
    
    return min_count, max_count



# Apply the function to extract min and max employee counts
company_df['min_employees'], company_df['max_employees'] = zip(*company_df['n_employees'].apply(extract_employee_counts))


# Function to calculate the number of days between two dates
def count_days_between_dates(start_date, end_date):
    start_d=start_date
    end_d =end_date

    # assuming if only start date is present 
    if pd.isna(end_d):
        return 1
    date_format = "%Y-%m-%d"  # Adjust the date format if needed
    start_d = datetime.strptime(start_d, date_format)
    end_d = datetime.strptime(end_d, date_format)
    days = abs((end_d - start_d).days)
    return days+1

event_df['event_active_days'] = event_df.apply(lambda row: count_days_between_dates(row['event_start_date'], row['event_end_date']), axis=1)

In [9]:
people_df["email_pattern"].value_counts()

[first].[last]            2024
[first_initial][last]      766
[first]                    468
[first][last]              140
[first]_[last]             119
[first_initial].[last]      81
[first][last_initial]       47
[last]                       1
Name: email_pattern, dtype: int64

In [10]:

def form_emails(row):

    if isinstance(row['email_pattern'], str):

        email_format = str(row['email_pattern'])

        # adding middle name pattern as well to cover future trend just incase

        placeholders = {
            '[first]': row['first_name'] if isinstance(row['first_name'], str) else '',
            '[last]': row['last_name'] if isinstance(row['last_name'], str) else '',
            '[middle]': row['middle_name'] if isinstance(row['middle_name'], str) else '',
            '[first_initial]': row['first_name'][0] if isinstance(row['first_name'], str) else '',
            '[last_initial]': row['last_name'][0] if isinstance(row['last_name'], str) else '',
            '[middle_initial]': row['middle_name'][0] if isinstance(row['middle_name'], str) else '',
        }

        for placeholder, value in placeholders.items():
            email_format = email_format.replace(placeholder, value)
        
        return f"{email_format}@{row['homepage_base_url']}".lower()
    
    else:
        return ""

people_df["email"] = people_df.apply(form_emails, axis=1) 

In [11]:
# get event industries

company_df["company_industry"].fillna("Other", inplace=True )

event_df['event_industry'] = event_df['event_url'].map(
    company_df.groupby('event_url')['company_industry'].agg(lambda x: ', '.join(set(x)))
)

event_df["event_industry"].fillna("Other", inplace=True )

event_df.head()


,event_logo_url,event_name,event_start_date,event_end_date,event_venue,event_country,event_description,event_url,event_active_days,event_industry
0,http://expo.overseaseducation.sg/v.fastcdn.co/...,World University Expo @ SUNTEC,2024-07-06,2024-07-06,SUNTEC Convention Centre,NaN,"Join our World University Expo this Saturday, ...",http://expo.overseaseducation.sg/,1,Other
1,https://ahiceconference.com/southeastasia/wp-c...,AHICE South East Asia,2025-02-25,2025-02-26,Pan Pacific Singapore,NaN,The Asian Hotel Industry Conference & Exhibiti...,https://ahiceconference.com/southeastasia/,2,"Professional Training and Coaching, Travel Arr..."
2,https://apac.commoditytradingweek.com/wp-conte...,Commodity Trading Week APAC,2025-02-25,2025-02-26,Marina Bay Sands,NaN,Commodity Trading Week APAC is the premier eve...,https://apac.commoditytradingweek.com/,2,"Maritime, Financial Services, Education Admini..."
3,https://architecturebuildingservices.com.sg/wp...,Architecture & Building Services (ABS) 2024,2024-11-13,2024-11-15,Marina Bay Sands Expo & Convention Centre Sing...,NaN,The 10th Architecture & Building Services (ABS...,https://architecturebuildingservices.com.sg/,3,"Wholesale Apparel and Sewing Supplies, Biotech..."
4,https://asiandownstreaminsights.com/wp-content...,Asian Downstream Summit,2024-10-23,2024-10-24,"Sands Expo & Convention Centre, Singapore",NaN,The Asian Downstream Summit is Asia’s most inf...,https://asiandownstreaminsights.com/events/asi...,2,Other


In [12]:

company_df.to_csv('processed_data/company_info.csv', index=False)
event_df.to_csv('processed_data/event_info.csv', index=False)
people_df.to_csv('processed_data/people_info.csv', index=False)